In [1]:
!pip install metpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import glob
import pandas as pd
from zipfile import ZipFile
from datetime import datetime
from metpy.io.metar import *
import re
import metpy

In [3]:
metpy.__version__

'1.6.2'

In [4]:
if 'data-science-challenge-at-eef-2024.zip' not in glob.glob('*'):
    !kaggle competitions download -c data-science-challenge-at-eef-2024

    with ZipFile('data-science-challenge-at-eef-2024.zip') as zip:
        zip.extractall()

In [5]:
df = pd.read_csv('public.csv')
print(df.shape)
df.head()

(302399, 10)


,flightid,hora_ref,origem,destino,url_img_satelite,metaf,metar,prev_troca_cabeceira,troca_cabeceira_hora_anterior,espera
0,504a62621cd231d6ab67e674ce538cd3,2022-06-01T01:00:00Z,SBCF,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0
1,b0fd0f83644625ecc21f5261e8e5e347,2022-06-01T01:00:00Z,SBPA,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0
2,1210f0ca07ddca00d09a3e02d3b100d8,2022-06-01T01:00:00Z,SBSP,SBCF,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCF 010000Z 12006KT CAVOK 21/14 Q1018=,0,0,0.0
3,b25032f34507cce285ee779446496568,2022-06-01T01:00:00Z,SBGR,SBCT,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCT 010000Z 10006KT 7000 -RA BKN004 OVC...,0,0,0.0
4,00762a9892ecba7c66d1d87800d38cac,2022-06-01T01:00:00Z,SBSP,SBSV,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBSV 010000Z 11008KT 9999 FEW023 27/21 Q...,0,1,0.0


In [6]:
# Criando uma variavel de Ano-Mes
df['data_ref'] = df['hora_ref'].apply(lambda x: x[:7])

## Tratando dados meteorologicos

Dados:  
- METAR (Meteorological Aerodrome Report) (Real)

- METAF (Terminal Aerodrome Forecast) (Previsto)

O METAR, sigla de Meteorological Aerodrome Report, é o informe meteorológico regular de aeródromo e contém as seguintes informações na sequência:

- Grupos de identificação;  
- Vento à superfície;  
- Visibilidade horizontal; 
- Alcance visual na pista (quando houver); 
- Tempo presente; 
- Nuvens (ou visibilidade vertical, se for o caso); 
- Temperaturas do ar e do ponto de orvalho; 
- Pressão atmosférica (QNH); e  
- Informações suplementares de inclusão condicional sobre tempo recente, cortante do vento, temperatura da superfície do mar, estado do mar e, por Acordo Regional de Navegação Aérea, o estado da pista.  

Grupo de Identificação 

METAR SBMN 061300Z 31015G27KT 280V350 5000 1500W -RA BKN010 SCT020 FEW025TCU 25/24 Q1014 RERA WS RWY17 W12/H75= 

METAR – Designador do código para informação meteorológica aeronáutica regular. Quando houver alguma mudança significativa fora do horário de confecção da mensagem METAR, o observador meteorológico deverá confeccionar a mensagem do tipo SPECI, sendo este campo substituído.

SBMN – código ICAO do aeroporto (S de América do Sul, B de Brasil e MN identificando o aeroporto de Manaus). 

061300Z – Dia do mês e horário da observação, em horas e minutos UTC, seguidos, sem espaço, da letra Z,  indicando o fuso horário ZULU. 



Alguns informes podem vir com a sigla COR apor o METAR, indicando uma correcao posterior daquele codigo, para esses casos os dados podem nao vir com a letra Z ao final do timestamp, portanto devemos tratar esse caso para decodificacao do informe.

In [7]:
# Preenchendo dados reais nulos por dados previstos
    # Alguns casos podem nao possuir tanto dados reais quanto previstos
df.loc[df['metar'].isna(), 'metar'] = df.loc[df['metar'].isna(), 'metaf']

# Substituir o texto METAF por METAR
df['metar'] = df['metar'].apply(lambda txt: re.sub('METAF', 'METAR', str(txt)))

In [8]:
# Funcao para ajustar dados meteorologicos com falta de informacao temporal
def metar_fix(txt):
    txt = str(txt)
    if txt != None and txt != 'nan':  
        first_position = txt.find('SB')
        second_position = txt[first_position+5:].find(' ')
        final_position = first_position+5+second_position
        if txt[final_position-1] != 'Z':
            txt = txt[:final_position] + 'Z' + txt[final_position:]
        return txt

df['metar'] = df['metar'].apply(metar_fix)

In [9]:
# Conversao do informe metar para uma tabela de dados
def convert_metar(df_met):
    df_final = pd.DataFrame()
    for i, row in df_met.iterrows():
        met, date = row['metar'], row['hora_ref']
        try:
            aux = parse_metar_to_dataframe(met, year=int(date[:4]), month=int(date[5:7]))
            # aux.drop('station_id', axis=1, inplace=True)
            # aux.reset_index(inplace=True)
            aux.dropna(axis=0, how='all', inplace=True)
            if aux is not None:
                aux['meta'] = met
                aux['date'] = date
                # return aux
                df_final = pd.concat([df_final, aux])
        except: 
            print(met, date)
            break
    return df_final
    
    
df_metar = convert_metar(df[['metar', 'hora_ref']].loc[~df['metar'].isna()].drop_duplicates())

C:\Users\pedro\AppData\Local\Temp\ipykernel_34408\1401959602.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, aux])
C:\Users\pedro\AppData\Local\Temp\ipykernel_34408\1401959602.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, aux])
C:\Users\pedro\AppData\Local\Temp\ipykernel_34408\1401959602.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this wil

In [10]:
metar = df_metar[df_metar.columns]
metar.isna().sum()

station_id                       0
latitude                         0
longitude                        0
elevation                        0
date_time                       47
wind_direction                3873
wind_speed                       1
wind_gust                    74620
visibility                       4
current_wx1                  64812
current_wx2                  73177
current_wx3                  75191
low_cloud_type               20023
low_cloud_level              21550
medium_cloud_type            46152
medium_cloud_level           46152
high_cloud_type              66447
high_cloud_level             66447
highest_cloud_type           74540
highest_cloud_level          74540
cloud_coverage                   0
air_temperature                 79
dew_point_temperature          169
altimeter                       76
current_wx1_symbol               0
current_wx2_symbol               0
current_wx3_symbol               0
remarks                          0
air_pressure_at_sea_

In [11]:
# Selecionando colunas que tenham no maximo 30% dos valores nulos para tratamento
valid_cols = metar.columns[(metar.isna().sum() <= metar.shape[0]*0.3).values]

metar_valid = metar[valid_cols]
metar_valid.dtypes

station_id                           object
latitude                            float64
longitude                           float64
elevation                             int64
date_time                    datetime64[ns]
wind_direction                      float64
wind_speed                          float64
visibility                          float64
low_cloud_type                       object
low_cloud_level                     float64
cloud_coverage                        int64
air_temperature                     float64
dew_point_temperature               float64
altimeter                           float64
current_wx1_symbol                    int64
current_wx2_symbol                    int64
current_wx3_symbol                    int64
remarks                              object
air_pressure_at_sea_level           float64
eastward_wind                       float64
northward_wind                      float64
meta                                 object
date                            

Por definicao da biblioteca utilizada, cloud_coverage é uma conversão numérica para low_cloud_type
Guia de conversão https://en.wikipedia.org/wiki/Okta

In [12]:
metar_valid.drop(['low_cloud_type', 'date_time'], axis=1, inplace=True)

C:\Users\pedro\AppData\Local\Temp\ipykernel_34408\497619564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metar_valid.drop(['low_cloud_type', 'date_time'], axis=1, inplace=True)


In [58]:
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

imputer = KNNImputer(n_neighbors=3)
scl = StandardScaler()

# Selecao das colunas validas para preenchimento
fixed_metar = metar_valid.drop(['meta', 'date', 'station_id', 'remarks'], axis=1).reset_index().drop(['station_id'], axis=1)


scl.fit(fixed_metar)

aux = pd.DataFrame(scl.transform(fixed_metar))

aux2 = pd.DataFrame(imputer.fit_transform(aux))
fixed_metar = pd.DataFrame(scl.inverse_transform(aux2), columns=fixed_metar.columns)

fixed_metar = pd.concat([metar_valid[['meta', 'date', 'station_id']].drop(['station_id'], axis=1).reset_index(), fixed_metar], axis=1)
fixed_metar.to_csv('metar.csv', index=False)